In [1]:
import torch

In [2]:
def fill_zero_places(a):
    a = torch.cat([torch.tensor([-1], device=a.device), a])
    nonzero_indices = torch.nonzero(a).squeeze()
    diff = torch.diff(torch.cat([nonzero_indices, torch.tensor([a.shape[0]], device=a.device)]))
    b = a[nonzero_indices].repeat_interleave(diff)
    b[b < 0] = 0
    c = b[1:]
    return c

In [3]:
def shift_values_to_next_available_zero(a):
    b = (a > 0).int()
    b = torch.cat([torch.tensor([0], device=a.device), b])
    c = torch.arange(b.shape[0], device=a.device) * b
    c[-1] = 0
    d = c[1:]
    e = torch.roll(fill_zero_places(d), 1)
    e[0] = 0
    f = torch.zeros_like(a)
    f.scatter_add_(0, e.long(), a)
    return f

In [4]:
def fill_zero_values_ascending_rest_with_zero(a):
    b = (a > 0).int()
    cumsum = torch.cumsum(1 - b, dim=0)
    c = (cumsum * (1 - b) - torch.maximum(b * cumsum, torch.zeros_like(cumsum)).cummax(dim=0)[0]) * (1 - b)
    return c

In [5]:
def find_one_zero_pattern(a):
    b = torch.roll(a.int(), -1)
    b[-1] = 1
    c = a - b
    d = (c == 1).int()
    return d

In [6]:
def fill_zero_values_descending_rest_with_zero(a):
    b = fill_zero_values_ascending_rest_with_zero(a)
    c = (b > 0).int()
    d = find_one_zero_pattern(c)
    e = d * b
    f = shift_values_to_next_available_zero(e)
    g = fill_zero_places(f)
    h = ((g + 1) - b) * c
    return h

In [7]:
def find_values_that_must_be_reordered(a):
    b, _ = torch.cummin(a.flip(0), dim=0)
    b = b.flip(0)
    c = torch.cat([(1 - (b[1:] - b[:-1]) > 0).int(), torch.tensor([0], device=a.device)])
    d = a * c
    e = ((d >= b).int() * c).int()
    f = (1 - c)
    g = fill_zero_values_descending_rest_with_zero(f)
    h = d - g
    i = ((h > b).int() * c | (a >= b).int() * c).int()
    return i

In [11]:
def adjust_cars_no_lane_change(a):
    b = a[-1] - a
    m = find_values_that_must_be_reordered(b).int()
    c = torch.where((torch.roll(m, 1) == 1) & (m == 0), torch.tensor(1, device=a.device), torch.tensor(0, device=a.device))
    d = (torch.cumsum(1 - c, dim=0) * (1 - c) - torch.maximum(c * torch.cumsum(1 - c, dim=0), torch.zeros_like(c)).cummax(dim=0)[0]) * (1 - c)
    e = a * c
    nonzero_indices = torch.nonzero(e, as_tuple=True)[0]
    repeat_counts = torch.diff(torch.cat((nonzero_indices, torch.tensor([e.shape[0]]))))
    f = e[nonzero_indices].repeat_interleave(repeat_counts)
    g = torch.roll(torch.argsort(e).cummax(dim=0)[0] * c, 1)
    h = torch.zeros_like(e)
    h.scatter_(0, g.long(), e)
    i = h.cummax(dim=0)[0]
    j = i - d
    k = fill_zero_values_descending_rest_with_zero(c)
    l = i - k
    n = 1 - m
    o = fill_zero_values_descending_rest_with_zero(n) * m
    p = (i - o) * m
    q = p + n * a
    return q

In [34]:
# bug
a = torch.tensor([ 83,  84,  89,  93,  87,  92,  90,  95,  90,  94,  94,  96,  97, 101,  97, 105, 104,  99, 101, 101, 101, 110, 105, 111, 106, 115, 90,  86, 107,  59])
adjust_cars_no_lane_change(a)

tensor([166, 168, 178, 186, 174, 184, 180, 190, 180, 188, 188, 192, 194, 202,
        194, 210, 208, 198, 202, 202, 202, 220, 210, 222, 212, 230, 180, 172,
        214, 118])

In [35]:
a = torch.tensor([ 3, 10,  8, 11,  5, 11,  6, 13, 11, 15, 16, 16, 12, 21, 16, 24, 19, 24, 23, 22, 28, 21, 27, 26, 33, 32, 30, 32, 31, 36])
adjust_cars_no_lane_change(a)

tensor([ 6, 20, 16, 22, 10, 22, 12, 26, 22, 30, 32, 32, 24, 42, 32, 48, 38, 48,
        46, 44, 56, 42, 54, 52, 66, 64, 60, 64, 62, 72])

In [36]:
a = torch.tensor([ 7,  4,  2,  6,  3,  5,  8,  6, 14,  4,  8, 13,  3,  2,  6,  7,  7,  17,  5,  3,  6, 23,  9, 23,  5,  6,  3, 10, 30, 35])
adjust_cars_no_lane_change(a)

tensor([14,  8,  4, 12,  6, 10, 16, 12, 28,  8, 16, 26,  6,  4, 12, 14, 14, 34,
        10,  6, 12, 46, 18, 46, 10, 12,  6, 20, 60, 70])

In [31]:
count = 30000
lane_num_tracks = torch.tensor([300000])
cars_position = torch.arange(0, count)
cars_speed = torch.randint(0,10, (count, 1)).flatten()

adjust_cars_no_lane_change(cars_position + cars_speed)

tensor([    4,    12,     4,  ..., 59994, 60014, 59998])

### Benchmarks

In [30]:
# import torch
# import time

# # Ensure CUDA is available
# if not torch.cuda.is_available():
#     raise RuntimeError("CUDA is not available, please run on a CUDA-capable device.")

# # Number of iterations
# iterations = 1000000
# count = 3000000

# lane_num_tracks = torch.tensor([300000000]).cuda()
# cars_position = torch.arange(0, count).cuda()
# cars_speed = torch.randint(0, 10, (count, 1)).flatten().cuda()

# def adjust_cars_no_lane_change(cars_info):
#     # Sample implementation, replace with the actual function
#     return cars_info * 2  # Example operation

# # Start timing
# start_time = time.time()

# # Execute the function multiple times
# for _ in range(iterations):
#     result = adjust_cars_no_lane_change(cars_position + cars_speed)

# # Optionally, synchronize and free the cache to get accurate timings
# torch.cuda.synchronize()
# torch.cuda.empty_cache()

# # End timing
# end_time = time.time()

# # Calculate elapsed time
# elapsed_time = end_time - start_time
# print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 178.74028062820435 seconds


Iterationen: 1.000.000
Autos: 300.000

Zeit: 16 Sekunden

Iterationen: 10.000.000
Autos: 30.000

Zeit: 94 Sekunden

Iterationen: 1.000.000
Autos: 3.000.000

Zeit: 178 Sekunden